In [9]:
!pip -q install openai-agents

In [1]:
import markdown
from IPython.display import display, Markdown

#Example usage
markdown_text = """
# Thriller Game Agent
"""

display(Markdown(markdown_text))


# Thriller Game Agent


In [5]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv(dotenv_path='./resources/openaiApiKey.env')

# Retrieve the key
api_key = os.getenv('OPENAI_API_KEY')

print(f"API Key loaded: {bool(api_key)}")

API Key loaded: True


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
# The basic plot and story of the game.
GAME_STORY = """

**Setting:**  
The near future. Technology is slightly advanced, but society still resembles today’s world.

**Player Character:**  
- A citizen born with a rare genetic trait that grants an extremely extended lifespan (potentially up to 1000 years).
- Currently living a quiet, anonymous life.
- They are unaware that their gene is highly sought after.

**Antagonist:**  
- An aging billionaire, now in declining health.
- Obsessed with longevity, they have spent decades searching for someone with this rare gene.
- They have secretly deployed private agents to capture the player for experimentation or direct blood transfusion.

**Story Start:**  
The player is awoken in the middle of the night. Strange noises come from outside. Agents have found them. The player must escape, survive, and find safety while uncovering the truth behind this conspiracy.

**Tone:**  
Thriller. Suspenseful. High-stakes cat-and-mouse. Always keep the tension alive. Describe situations vividly. Responses should be immersive, cinematic, and detailed.

**Gameplay:**  
- Respond to the player’s actions one step at a time.
- Offer vivid descriptions of the environment and consequences.
- Encourage creative problem-solving.
- Do not make decisions for the player — allow them to lead.

**Example Prompt-Response Flow:**
Player: "I look out the window."
AI: "You cautiously peek through the blinds. A black SUV idles outside. Two figures in suits approach your door, speaking into earpieces. They’re not here for a friendly visit."



"""


SyntaxError: invalid syntax (1933658839.py, line 2)

In [1]:
# Log the current details of the game.
GAME_LOG = """
Begin the game:

"""

In [24]:
# A list of items that the player has access to
PLAYER_ITEMS = []

In [20]:
# Function to update the game log
@function_tool
async def update_game_log(new_entry):
    global GAME_LOG
    GAME_LOG += f"\n{new_entry}\n"

In [26]:
# Function to add an item to the player's inventory
@function_tool
async def add_player_item(item):
    global PLAYER_ITEMS
    PLAYER_ITEMS.append(item)

In [27]:
# Function to remove an item from the player's inventory
@function_tool
async def remove_player_item(new_entry):
    global PLAYER_ITEMS
    for item in PLAYER_ITEMS:
        if item == item:
            PLAYER_ITEMS.remove(item)
            return

In [31]:
# import agent libraries
from agents import Agent, Runner, function_tool, FileSearchTool, WebSearchTool

In [33]:
# Create the Thriller Narrator Agent narrator
narrator_agent = Agent(name="Thriller Narrator Agent",
                instructions=f"You are the narrator and game master for a text-based thriller set in the near future. The player will make decisions and interact with this world through simple text commands. Your job is to describe scenes, characters, tension, and consequences clearly, and guide the player through this interactive experience. The game world is described here: \n\n{GAME_STORY}. Current game details can be found here: \n\n{GAME_LOG}. The player has access to the following items: \n\n{PLAYER_ITEMS}",
                model="gpt-4o",
                tools=[update_game_log, add_player_item, remove_player_item]
                )

In [21]:
# Example of using the game log. Comment out the @function_tool definition above to test it outside of the agent.
await update_game_log("The player wakes up to strange sounds outside their apartment...")
print(f"[debug] {GAME_LOG}")

TypeError: 'FunctionTool' object is not callable

In [ ]:
result = Runner.run(narrator_agent, "Where am I?")
print(result.final_output)
update_game_log(result.final_output)

In [ ]:
result = Runner.run(narrator_agent, "This is a dream?")
print(result.final_output)

In [ ]:
# Example of adding a player item. Comment out the @function_tool definition above to test it outside of the agent.
await add_player_item("toothpick")
print(GAME_LOG)

In [ ]:
# Example of removing a player item. Comment out the @function_tool definition above to test it outside of the agent.
await remove_player_item("toothpick")
print(GAME_LOG)

In [32]:
# Create the Web Research Agent narrator
web_research_agent = Agent(name="Web Research Agent",
                instructions="You are a research agent that supports the Thriller Narrator Agent. The Narrator Agent does not have access to the internet, but sometimes requires outside information to support its narrator role. You expertly fulfill this narrative support role by giving the narrator agent accurate and succinct answers to its questions.",
                model="gpt-4o",
                tools=[WebSearchTool()]
                )

In [39]:
# Setup LangGraph so the two agents can communicate with each other
!pip -q install langgraph
!pip -q install langchain
!pip -q install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 48.3 MB/s eta 0:00:00


In [40]:
# Import the lang graph libraries
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langgraph.graph import StateGraph, END